In [58]:
#Import required packages

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import numpy as np # linear algebra
from sklearn.model_selection import train_test_split

In [45]:
trainx = pd.read_csv("trainx.csv")
trainy = pd.read_csv("trainy.csv")
testx = pd.read_csv("testx.csv")
testy = pd.read_csv("testy.csv")

trainx = trainx.loc[:, ~trainx.columns.str.contains('^Unnamed')]
trainy = trainy.loc[:, ~trainy.columns.str.contains('^Unnamed')]
testx = testx.loc[:, ~testx.columns.str.contains('^Unnamed')]
testy = testy.loc[:, ~testy.columns.str.contains('^Unnamed')]

X = trainx.head(100000).sample(frac=1)
Y = trainy.head(100000).sample(frac=1)

In [62]:
data = pd.read_csv("wh.csv")
data = data.drop(['Formatted Date','Summary','Daily Summary','Loud Cover'],axis=1)
data['precip'] = np.where(data['Precip Type']!= 'snow', 1, 0)

X = np.array(data.drop(['Precip Type','precip'],axis=1)) #independent variable
Y = np.array(data['precip']) #target variable
trainx, testx, trainy, testy = train_test_split(X, Y, test_size=0.4, random_state=1999)

In [70]:
DecisionTreeClassifier().__init__()
clf = DecisionTreeClassifier()
#clf = clf.__init__(max_depth=20)

#This line doesnt seem to stop running on the full dataset?
clf = clf.fit(trainx, trainy)
clf.score(testx, testy)

predicty = clf.predict(testx)
predY = pd.DataFrame(data=predicty)

testy = pd.DataFrame(data=testy)
df= pd.concat([predY, testy], axis=1)
df.head(100)

,0,0
0,1,1
1,0,0
2,1,1
3,1,1
4,0,0
...,...,...
95,1,1
96,1,1
97,1,1
98,1,1


In [47]:
clf.score(testx, testy)

0.5074127831631331

In [48]:
predictx = clf.predict(testx)
predX = pd.DataFrame(data=predictx)
print(predX)

       0
0      0
1      0
2      0
3      1
4      1
...   ..
38577  0
38578  1
38579  1
38580  0
38581  1

[38582 rows x 1 columns]


In [49]:
print(testy)

       Precip Type
0                1
1                0
2                1
3                1
4                0
...            ...
38577            1
38578            0
38579            1
38580            1
38581            1

[38582 rows x 1 columns]


In [50]:
print(testx)

       Temperature (C)  Apparent Temperature (C)  Humidity  Wind Speed (km/h)  \
0             2.105556                 -2.572222      0.67            20.0284   
1             0.000000                 -1.583333      0.74             5.0232   
2            22.800000                 22.800000      0.53             3.7996   
3            18.800000                 18.800000      0.93             3.3327   
4            -2.400000                 -2.400000      0.88             4.6046   
...                ...                       ...       ...                ...   
38577         3.722222                  1.266667      0.84             9.4990   
38578        -2.916667                 -2.916667      0.86             1.4812   
38579        28.977778                 27.788889      0.29             7.5348   
38580        19.238889                 19.238889      0.78            11.2700   
38581        17.150000                 17.150000      0.67            17.3236   

       Wind Bearing (degree

In [51]:
print(testy)

       Precip Type
0                1
1                0
2                1
3                1
4                0
...            ...
38577            1
38578            0
38579            1
38580            1
38581            1

[38582 rows x 1 columns]


In [52]:
print(trainx)

        Temperature (C)  Apparent Temperature (C)  Humidity  \
0             20.438889                 20.438889  0.880000   
1             14.705556                 14.705556  0.430000   
2              7.755556                  5.327778  0.740000   
3             27.916667                 27.994444  0.460000   
4             10.050000                 10.050000  0.770000   
...                 ...                       ...       ...   
102951        -3.173606                 -9.479514  0.798467   
102952        -6.058575                 -6.058575  0.977446   
102953        -4.051473                 -6.790153  0.881863   
102954        -1.447975                 -3.801885  0.899674   
102955        -0.854198                 -3.127161  0.874884   

        Wind Speed (km/h)  Wind Bearing (degrees)  Visibility (km)  \
0                3.075100              140.000000        14.763700   
1               20.334300              340.000000        10.577700   
2               13.829900        

In [53]:
print(trainy)

        Precip Type
0                 1
1                 1
2                 1
3                 1
4                 1
...             ...
102951            0
102952            0
102953            0
102954            0
102955            0

[102956 rows x 1 columns]


In [54]:
t = pd.DataFrame({'a':20, 'b':20, 'c':0.85, 'd':3, 'e':140, 'f':14.5, 'g':1000}, index=[0])
z = clf.predict(t)

print(z)

[0]
